## Notebook to demonstrate Data-Services workflow
### The workflow in a nutshell
TAO Data Services include 4 key pipelines:
1. Offline data augmentation using DALI
2. Auto labeling using TAO Mask Auto-labeler (MAL)
3. Annotation conversion
4. Groundtruth analytics

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Convert KITTI dataset to COCO format
* Run auto-labeling to generate pseudo masks for KITTI bounding boxes
* Apply data augmentation to the KITTI dataset with bounding boxe refinement
* Run data analytics to collect useful statistics on the original and augmented KITTI dataset

### Table of contents

1. [Convert KITTI data to COCO format](#head-1)
2. [Generate pseudo-masks with the auto-labeler](#head-2)
3. [Apply data augmentation](#head-3)
4. [Perform data analytics](#head-4)


### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
# imports
import json
import os
import requests
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME 1
2. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
3. Assign the ngc_api_key variable in FIXME 3
4. Assign path to save the dataset in FIXME 4

In [ ]:
workdir = "<workdir_data_services>"  # FIXME 1
host_url = "http://<ip_address>:<port_number>"  # FIXME 2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port_number can be obtained as follows,
# ip_address: hostname -I
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>"  # FIXME 3 example: (Add NGC API key) 
data_dir = "<dataset_path>"    # FIXME 4
job_map = {}

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID", user_id)
token = response.json()["token"]
print("JWT", token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to", base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

## 1. Convert KITTI data to COCO format <a class="anchor" id="head-1"></a>
We would first convert the dataset from KITTI to COCO formats.

### Create the dataset
We support both KITTI and COCO data formats

KITTI dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── labels
    ├── image_name_1.txt
    ├── image_name_2.txt
    ├── ...
```

And COCO dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── annotations.json
```
For this notebook, we will be using the kitti object detection dataset for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d).

In [ ]:
# Dataset Links
images_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip"
labels_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip"

In [ ]:
# Download the dataset
!wget -O images.zip {images_url}
!wget -O labels.zip {labels_url}

In [ ]:
!unzip -q images.zip -d {data_dir}/
!unzip -q labels.zip -d {data_dir}/
!mkdir -p {data_dir}/images {data_dir}/labels
!mv {data_dir}/training/image_2/000* {data_dir}/images/
!mv {data_dir}/training/label_2/000* {data_dir}/labels/
!cd {data_dir} && tar -cf kitti_dataset.tar images labels
!rm -rf images.zip labels.zip {data_dir}/training/ {data_dir}/training/ {data_dir}/testing/

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "kitti"
data = json.dumps({"type": ds_type, "format": ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
kitti_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{kitti_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
dataset_path = f"{data_dir}/kitti_dataset.tar"
files = [("file", open(dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{kitti_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets

In [ ]:
# List the created datasets
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(f"{rsp['id']}\t{rsp['type']}\t{rsp['format']}\t\t{rsp['name']}")

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "annotations"
action = "convert"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/model"

response = requests.post(endpoint, data=data, headers=headers)

model_id = response.json()["id"]
print(model_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": kitti_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/{action}/schema"
 
response = requests.get(endpoint, headers=headers)

print(response)
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Updating spec file
specs["data"]["input_format"] = "KITTI"
specs["data"]["output_format"] = "COCO"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/{action}"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Execute the data format conversion action 


In [ ]:
# Run action
parent = None
actions = [action]
data = json.dumps({"job":parent, "actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
convert_job_id = job_map[action]
endpoint = f"{base_url}/model/{model_id}/job/{convert_job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
       
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
convert_job_id = job_map[action]
endpoint = f'{base_url}/model/{model_id}/job/{convert_job_id}/download'

# Save
temptar = f'{convert_job_id}.tar.gz'
with requests.get(endpoint, stream=True, headers=headers) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{convert_job_id}")
convert_out_path = f"{workdir}/{convert_job_id}"

## 2. Generate pseudo-masks with the auto-labeler <a class="anchor" id="head-2"></a>
Here we will use a pretrained MAL model to generate pseudo-masks for the converted KITTI data. 

### Create the dataset
We would be formatting the original dataset to include the COCO annotations generated.


In [ ]:
# Reformatting the dataset
# Untar to destination
tar_command = f'mkdir -p {workdir}/{convert_job_id}_coco/ && tar -xf {dataset_path} -C {workdir}/{convert_job_id}_coco/'
os.system(tar_command)

# Copy the annotations
copy_command = f'cp {convert_out_path}/{kitti_dataset_id}.json {workdir}/{convert_job_id}_coco/annotations.json'
os.system(copy_command)

# Tar the dataset
tar_command = f'cd {workdir} && tar -cf {convert_job_id}_coco.tar {convert_job_id}_coco'
os.system(tar_command)
coco_data = f'{workdir}/{convert_job_id}_coco.tar'

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type": ds_type,"format": ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
coco_dataset_id = response.json()["id"]

In [ ]:
# Update dataset information
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{coco_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(coco_data, "rb"))]

endpoint = f"{base_url}/dataset/{coco_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### Create the model

In [ ]:
# Defining the task
network_arch = "auto_label"
action = "generate"

data = json.dumps({"network_arch": network_arch})
print(data)
endpoint = f"{base_url}/model"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]
print(model_id)

### Find the PTM

In [ ]:
# List models
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    if rsp["name"] == "Mask Auto Label":
        print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
pretrained_map = {"auto_label" : "mask_auto_label:trainable_v1.0"}

In [ ]:
# Get pretrained model
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm = []
for rsp in response_json:
    if rsp["network_arch"] == network_arch and rsp["ngc_path"].endswith(pretrained_map[network_arch]):
        ptm_id = rsp["id"]
        ptm = [ptm_id]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break

### Assign the dataset

In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": coco_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign the PTM

In [ ]:
# Assign PTM
dataset_information = {"ptm": ptm}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/{action}/schema"

response = requests.get(endpoint, headers=headers)

print(response)
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
specs["gpu_ids"] = [0]

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/{action}"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Execute the auto labeling action

In [ ]:
# Run action
parent = None
actions = [action]
data = json.dumps({"job": parent, "actions": actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
label_job_id = job_map[action]
endpoint = f"{base_url}/model/{model_id}/job/{label_job_id}"

while True: 
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
label_job_id = job_map[action]
endpoint = f'{base_url}/model/{model_id}/job/{label_job_id}/download'

# Save
temptar = f'{label_job_id}.tar.gz'
with requests.get(endpoint, stream=True, headers=headers) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{label_job_id}")
inference_out_path = f"{workdir}/{label_job_id}"

## 3. Apply data augmentation <a class="anchor" id="head-3"></a>
In this section, we run offline augmentation with the original dataset. During the augmentation process, we can use the pseudo-masks generated from the last step to refine the distorted or rotated bounding boxes

### Create the dataset
We would be formatting the dataset to include the generated mask information.

In [ ]:
# Format the dataset
copy_command = f'cp {workdir}/{label_job_id}/label.json {workdir}/{convert_job_id}_coco'
os.system(copy_command)

# Tar the dataset
tar_command = f'cd {workdir} && tar -cvf {label_job_id}_coco.tar {convert_job_id}_coco'
os.system(tar_command)
coco_data = f'{workdir}/{label_job_id}_coco.tar'

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type": ds_type,"format": ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
coco_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{coco_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(coco_data, "rb"))]

endpoint = f"{base_url}/dataset/{coco_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "augmentation"
action = "generate"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/model"

response = requests.post(endpoint, data=data, headers=headers)

model_id = response.json()["id"]
print(model_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": coco_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/{action}/schema"

response = requests.get(endpoint, headers=headers)

print(response)
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/{action}"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Execute the data augmentation action


In [ ]:
# Run action
parent = None
actions = [action]
data = json.dumps({"job":parent, "actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
augment_job_id = job_map[action]
endpoint = f"{base_url}/model/{model_id}/job/{augment_job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
augment_job_id = job_map[action]
endpoint = f'{base_url}/model/{model_id}/job/{augment_job_id}/download'

# Save
temptar = f'{augment_job_id}.tar.gz'
with requests.get(endpoint, stream=True, headers=headers) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{augment_job_id}")
inference_out_path = f"{workdir}/{augment_job_id}"

## 4. Perform data analytics  <a class="anchor" id="head-4"></a>
Next, we perform analytics with the KITTI dataset.

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "analytics"
action = "analyze"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/model"

response = requests.post(endpoint, data=data, headers=headers)

model_id = response.json()["id"]
print(model_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": kitti_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/{action}/schema"
 
response = requests.get(endpoint, headers=headers)

print(response)
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/{action}"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Execute the data analytics action


In [ ]:
# Run action
parent = None
actions = [action]
data = json.dumps({"job":parent, "actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map[action]
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
       
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
job_id = job_map[action]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, stream=True, headers=headers) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"